In [74]:
from pymongo import MongoClient

# connect to mongo and clear existing collection
client = MongoClient('mongodb://localhost:27017/')
db = client['your_database']
kafka_aggregated_data = db['kafka_aggregated_data'] #prosessed agregated data from kafka
kafka_raw_data = db['kafka_raw_data'] #unprosessed raw data from kafka broker

In [169]:
def find_link_with_least_vehicles(dt_start = '2025-02-09 22:50:00', dt_end= '2025-02-09 22:55:00'):
    collection = kafka_aggregated_data

    # result = collection.find(query)#, sort = [("total_vehicles", -1)]
    result = collection.aggregate([
        {
            # First stage: Match documents within the time range
            '$match': {
                'window_start': {'$gte': dt_start},
                'window_end': {'$lte': dt_end}
            }
        },
        {
            # Second stage: Group by link and sum vehicles
            '$group': {
                '_id': '$link',              # Group by name
                'total_vehicles': {'$sum': '$total_vehicles'}, # Calculate sum of age
            }
        },
        {
            # Optional: Sort by total vehicles (ascending for least vehicles)
            '$sort': {
                'total_vehicles': 1
            }
        }
    ])
    result = list(result)[0]
    return result

def find_link_with_max_avg_vehicle_speed(dt_start = '2025-02-09 22:50:00', dt_end= '2025-02-09 22:55:00'):
    collection = kafka_aggregated_data
 
    # result = collection.find(query)#, sort = [("total_vehicles", -1)]
    result = collection.aggregate([
        {
            
            '$match': {
                'window_start': {'$gte': dt_start},
                'window_end': {'$lte': dt_end}
            }
        },
        {
            
            '$group': {
                '_id': '$link',              
                'average_speed': {'$avg': '$average_speed'}, 
            }
        },
        {
            # -1 = desc, 1 = asc
            '$sort': {
                'average_speed': -1
            }
        }
    ])
    result = list(result)[0]
    return result
    

def find_the_longest_route(dt_start = '2025-02-09 22:50:00', dt_end= '2025-02-09 22:55:00'):
    collection = kafka_raw_data

    # result = collection.find(query)#, sort = [("total_vehicles", -1)]
    result = collection.aggregate([
        {
            # First stage: Match documents within the time range
            '$match': {
                'time': {'$gte': dt_start, '$lte': dt_end}
            }
        },
        {
            
            '$group': {
                '_id': {
                    'name': '$name',
                    'link': '$link'
                },
                'vehicle_min_pos_in_link': {'$min': '$position'}, 
                'vehicle_max_pos_in_link': {'$max': '$position'}
            }
        },
        {
            '$addFields': {
                'vehicle_distance_traveled_in_link': {
                    '$subtract': ['$vehicle_max_pos_in_link', '$vehicle_min_pos_in_link']
                }
            }
        },
        {
            
            '$group': {
                '_id': '$_id.name', # Reference the name from the previous grouping        
                'vehicle_distance_traveled_total': {'$sum': '$vehicle_distance_traveled_in_link'}, 
            }
        },
        {
            '$sort': {
                'vehicle_distance_traveled_total': -1
            }
        }
    ])

    result = list(result)[0]
    return result

dt_start = '2025-02-10 01:44:55'; dt_end= '2025-02-10 01:49:55'
print('results for time frame =[', dt_start, '/' ,dt_end, ']')

out_1 = find_link_with_least_vehicles(dt_start, dt_end)
print(f'link with least vehicles {out_1["_id"]}, vehicles in link {out_1["total_vehicles"]}')
# print(f"Total users found: {len(out_1)}")
# for p in out_1:
#     print(p)
    
out_2 = find_link_with_max_avg_vehicle_speed(dt_start, dt_end)
print(f'link with max avg vehicle speed {out_2["_id"]}, vehicles avg speed {out_2["average_speed"]} m/s')

# print(f"Total users found: {len(out_2)}")
# for p in out_2:
#     print(p)   
    
out_3 = find_the_longest_route(dt_start, dt_end)
print(f'longest distnce traveled by vehicle number {out_3["_id"]}, total distance {out_3["vehicle_distance_traveled_total"]} m')
# print(f"Total users found: {len(out_3)}")
# for p in out_3:
#     print(p)  

 
# collection = kafka_raw_data
# out = collection.aggregate([
#         {
#             # First stage: Match documents within the time range
#             '$match': {
#                 'name': '27'
#             }
#         }
#     ])
# out = list(out)
# for p in out:
#     print(p) 

results for time frame =[ 2025-02-10 01:44:55 / 2025-02-10 01:49:55 ]
link with least vehicles I3S3, vehicles in link 15
link with max avg vehicle speed I1W1, vehicles avg speed 45.89411976911977 m/s
longest distnce traveled by vehicle number 102, total distance 1833.3333333333333 m


In [165]:
# list(kafka_raw_data.find())
# list(kafka_aggregated_data.find())


[{'_id': '2025-02-10 01:44:50_2025-02-10 01:45:00_N3I3',
  'average_speed': 30.0,
  'link': 'N3I3',
  'total_vehicles': 1,
  'window_end': '2025-02-10 01:45:00',
  'window_start': '2025-02-10 01:44:50'},
 {'_id': '2025-02-10 01:44:50_2025-02-10 01:45:00_E1I4',
  'average_speed': 50.0,
  'link': 'E1I4',
  'total_vehicles': 1,
  'window_end': '2025-02-10 01:45:00',
  'window_start': '2025-02-10 01:44:50'},
 {'_id': '2025-02-10 01:44:50_2025-02-10 01:45:00_N1I1',
  'average_speed': 30.0,
  'link': 'N1I1',
  'total_vehicles': 1,
  'window_end': '2025-02-10 01:45:00',
  'window_start': '2025-02-10 01:44:50'},
 {'_id': '2025-02-10 01:44:50_2025-02-10 01:45:00_S2I2',
  'average_speed': 30.0,
  'link': 'S2I2',
  'total_vehicles': 1,
  'window_end': '2025-02-10 01:45:00',
  'window_start': '2025-02-10 01:44:50'},
 {'_id': '2025-02-10 01:44:50_2025-02-10 01:45:00_S4I4',
  'average_speed': 30.0,
  'link': 'S4I4',
  'total_vehicles': 1,
  'window_end': '2025-02-10 01:45:00',
  'window_start': '202